In [1]:
import torch
import json
import os
import numpy as np
import mlflow
import pickle
import time
import torch.optim as optim
from src.torch_dataset import ZootrDataset 
from src.mlflow_functions import start_mlflow
from src.pytorch_efficientnet import augment, augmented_pred
from src.train_test_functions import train_model


In [2]:
start_mlflow('zootr_img_id')

In [3]:
run_id = '28997c1fcb5846088497214958a53687'

with open(r'mlartifacts\873592191447312121\{}\artifacts\loss_dict_collection.json'.format(run_id), 'r') as file:
    loss_dict_collection = json.load(file)

with open(r'mlartifacts\873592191447312121\{}\artifacts\train_objects.pkl'.format(run_id), 'rb') as file:
    train_objs = pickle.load(file)

#### Determine best epoch

In [4]:
best_val_loss = float('inf')
best_epoch = None
for epoch in loss_dict_collection.keys():
    loss_dict = loss_dict_collection[epoch]
    val_loss = loss_dict['val']
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch

### Retrain model with the entire data

In [5]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
NR_EPOCHS = int(best_epoch)

In [6]:
optimizer = train_objs['optimizer']
scheduler = train_objs['scheduler']
criterion = train_objs['criterion']
model = train_objs['model']

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
#Get all items
non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
#Get non items
all_images = list(np.random.permutation(items_images + non_items_images))

In [9]:
data = {'train': all_images, 
        'val': all_images[0:10], 
        'test': all_images[0:10]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': len(all_images), 'val': 1, 'test': 1}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}

dics = {
    'dataloaders': dataloaders,
    'dataset_sizes': dataset_sizes
}

In [ ]:
time.sleep(10)
#Useful to dodge memory problems

In [11]:
mdl, loss_dict_collection = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=NR_EPOCHS)

Epoch 1


100%|██████████| 103/103 [01:11<00:00,  1.44it/s]
0it [00:00, ?it/s]


Train loss: 0.3271673289571457 Val loss: 0.0
Epoch 2


100%|██████████| 103/103 [01:09<00:00,  1.49it/s]
0it [00:00, ?it/s]


Train loss: 0.12701486536974368 Val loss: 0.0
Epoch 3


100%|██████████| 103/103 [01:11<00:00,  1.43it/s]
0it [00:00, ?it/s]


Train loss: 0.07017688300365056 Val loss: 0.0
Training complete in 3m 33s
Best val loss: 0.000000
